In [1]:
from Tkinter import *
from bs4 import BeautifulSoup
import urllib2
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import pymongo
from numpy import arange


In [2]:
class LiveData:
    def __init__(self):
        self.top_movies_result = ""
        self.top_actors_result = ""

    #Get top movies.  Open url, use beautiful soup, and parse html code. Populate database.
        
    def get_top_movies(self):
        if self.top_movies_result == "":
            response = urllib2.urlopen('http://www.boxofficemojo.com/alltime/domestic.htm')
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            self.top_movies_result = {}
            for row in soup.find_all('tr'):
                if row.get('bgcolor') in ("#ffffff", "#f4f4ff", "#FFFF99", "#ffff99"):
                    columns = row.find_all("td")
                    if int(columns[0].get_text()) > 10:
                        break
                    self.top_movies_result[int(columns[0].get_text())] = {'name': columns[1].get_text(),
                                                                          'gross': columns[3].get_text(),
                                                                          'year': int(columns[4].get_text()[:4]),
                                               'link': 'http://www.boxofficemojo.com'+columns[1].find('a').get('href')}
        return self.top_movies_result

    #Get top actors.  Open url, use beautiful soup, and parse html code. Populate database.    
    
    def get_top_actors(self):
        if self.top_actors_result == "":
            response = urllib2.urlopen('http://www.boxofficemojo.com/people/?view=Actor&sort=sumgross&p=.htm')
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')
            self.top_actors_result = {}
            for row in soup.find_all('tr'):
                if row.get('bgcolor') in ("#ffffff", "#f4f4ff", "#FFFF99"):
                    columns = row.find_all("td")
                    if int(columns[0].get_text()) > 10:
                        break
                    self.top_actors_result[int(columns[0].get_text())] = {'name': columns[1].get_text(),
                                                                          'gross': columns[2].get_text() + " mi",
                                                'link': 'http://www.boxofficemojo.com'+columns[1].find('a').get('href')}
        return self.top_actors_result


    #Search movies.  Goes pyhsically to website and parses data.  Returns the structure as JSON.
   
    
    def get_movie(self, name):
        response = urllib2.urlopen('http://www.boxofficemojo.com/search/?q='+name.replace(" ", "%20"))
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        if soup.find("b").get_text() == "No Movies or People found.":
            return "No Movies Found!"
        elif soup.find("b").get_text() != "1 Movie Matches: ":
            return "More than one movie was found, please refine your search."
        else:
            for row in soup.find_all("tr"):
                if row.get('bgcolor') in ("#FFFF99", "#FFFFFF", "#F4F4FF"):
                    movie_link = row.find("a").get("href")
                    response2 = urllib2.urlopen('http://www.boxofficemojo.com'+movie_link)
                    html2 = response2.read()
                    soup2 = BeautifulSoup(html2, 'html.parser')
                    tr = soup2.find_all("tr")
                    for tag in tr:
                        if tag.get("bgcolor") == "#ffffff":
                            font = tag.find_all("font")
                            for f in font:
                                if f.get("size") == "4":
                                    gross = int(f.find("b").get_text().replace("$", "").replace(",", ""))
                                elif f.get("size") == "2":
                                    gross = int(f.find("b").get_text()[25:].replace("$", "").replace(",", ""))
                                    break
                            break
                    year = soup2.title.string.split("(")[1][:4]
                    break

            myfont = soup2.findAll("font", { "size": "2" })
            actors = []

        #Get actors from movie. If more than 10 actors stop or if parsed info is not an actor then stop. 
            
            actors_check = False
            producers_check = False
            for font in myfont:
                if len(actors) > 10:
                    break
                links = font.find_all("a")
                for link in links:
                    if actors_check:
                        if link.get_text() == "Producers:" or "Actor" not in link.get("href"):
                            producers_check = True
                            break
                        response3 = urllib2.urlopen('http://www.boxofficemojo.com'+link.get("href"))
                        html3 = response3.read()
                        soup3 = BeautifulSoup(html3, 'html.parser')
                        all_gross = soup3.findAll("font", {"face": "Verdana", "size": "2"})
                        for g in all_gross:
                            if "Average" in g.get_text():
                                actor_gross = g.get_text().split(": ")[1].split("Average")[0].replace("$", "").replace(",", "")
                                actors.append({"name": link.get_text(),
                                               "link": 'http://www.boxofficemojo.com'+link.get("href"),
                                               "gross": actor_gross})
                                break
                    else:
                        if link.get_text() == "Actors:":
                            actors_check = True
                if producers_check:
                    break

            result = {"name": name.title(), "gross": gross, "year": year, "actors": actors}
            return result

    #Search actors, display and store in database.    
        
    def get_actor(self, name):
        response = urllib2.urlopen('http://www.boxofficemojo.com/search/?q='+name.replace(" ", "%20"))
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        if soup.find("b").get_text() == "No Movies or People found.":
            return "No Actors found."
        elif soup.findAll("b")[1].get_text()[:13] != "Person Match:":
            return "More than one actor was found, please refine your search."
        else:
            for row in soup.find_all("td"):
                if row.get('bgcolor') in ("#FFFF99", "#ffff99", "#FFFFFF", "#F4F4FF"):
                    actor_link = row.find("a").get("href")
                    response2 = urllib2.urlopen('http://www.boxofficemojo.com'+actor_link)
                    html2 = response2.read()
                    soup2 = BeautifulSoup(html2, 'html.parser')
                    all_gross = soup2.findAll("font", {"face": "Verdana", "size": "2"})
                    for g in all_gross:
                        if "Average" in g.get_text():
                            gross = g.get_text().split(": ")[1].split("Average")[0].replace("$", "").replace(",", "")
                            break

                    mytable = soup2.findAll("table", { "border": "0", "cellspacing": "1", "cellpadding": "5" })
                    movies = []
                    mytr = mytable[0].findAll("tr")
                    
                    #If actor in more than 10 movies then stop.
                    
                    for tr in mytr:
                        if len(movies) == 10:
                            break
                        if tr.get("bgcolor") in ("#ffff99", "#f4f4ff", "#ffffff"):
                            mytd = tr.findAll("td")
                            year = int(mytd[0].get_text()[-2:])
                            if year > 50:
                                year = int("19"+mytd[0].get_text()[-2:])
                            else:
                                year = int("20"+mytd[0].get_text()[-2:])
                            movie_name = mytd[1].get_text()
                            movie_gross = mytd[3].get_text()
                            if movie_gross == "n/a":
                                movie_gross = 0
                            else:
                                movie_gross = int(movie_gross.replace("$", "").replace(",", ""))
                            movies.append({"name": movie_name, "gross": movie_gross, "year": year,
                                           "link": 'http://www.boxofficemojo.com'+mytd[1].find("a").get("href")})

            result = {"name": name.title(), "gross": gross, "movies": movies}
            return result



In [3]:
#Connecting to MongoDB, search and write

class DatabaseAccess:
    def __init__(self):
        client = pymongo.MongoClient()
        self.db = client.vicecity

    def search_movie(self, name):
        collection = self.db.movies
        result = collection.find_one({"name": name.title()})
        if result == None:
            return False
        else:
            return result

    def search_actor(self, name):
        collection = self.db.actors
        result = collection.find_one({"name": name.title()})
        if result == None:
            return False
        else:
            return result

    def insert_movie(self, movie):
        self.db.movies.insert_one(movie)

    def insert_actor(self, actor):
        self.db.actors.insert_one(actor)


In [4]:
#Create GUI

class ProjectHollywood:
    def __init__(self):
        # Initializing the window
        self.master = Tk()
        self.master.title("Project Hollywood")  # Window title
        self.master.geometry('1400x800')  # Window size

        # Frame same size as window
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Call welcome page
        self.welcome_page()

    def welcome_page(self):
        # Destroy old frame in case we come back from second page
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Creating titles
        t0 = Label(self.frame, text=" ")  # Empty label to add blank space in the beginning
        t1 = Label(self.frame, text="Welcome to", font=("", 50, "bold"))
        t2 = Label(self.frame, text="VICE CITY", font=("", 50, "bold"))
        t3 = Label(self.frame, text=" ")  # Empty label to add blank space in the end
        t0.pack(pady=50)  # Defining the size of the space
        t1.pack(pady=25)
        t2.pack()
        t3.pack(pady=100)  # Defining the size of the space

        # Buttons at the end of the page
        b1 = Button(self.frame, text="Goodbye", command=self.goodbye, width=15, height=3)
        b2 = Button(self.frame, text="Enter", command=self.enter, width=15, height=3)
        # Aligning the buttons on the same line
        b1.pack(side=LEFT, padx = 200)
        b2.pack(side=RIGHT, padx = 200)

        mainloop()

    def goodbye(self):
        # Clear the frame to show new information
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # In case of goodbye screen, show thank you message
        t0 = Label(self.frame, text=" ")
        t1 = Label(self.frame, text="Thank you", font=("", 50, "bold"))
        t2 = Label(self.frame, text="for visiting!", font=("", 50, "bold"))
        t0.pack(pady=50)
        t1.pack(pady=25)
        t2.pack()

        # Close the window after 5 seconds
        self.master.after(5000, lambda: self.master.destroy())

    def enter(self):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY", font=("", 40, "bold"))
        t2 = Label(self.frame, text=" ")
        t1.pack(pady=25)
        t2.pack(pady=45)

        # List of options
        l1 = Label(self.frame, text="1. Top 10 grossing movies of all time", font=("", 16), cursor="pointinghand")
        l2 = Label(self.frame, text="2. Top 10 actors of all time", font=("", 16), cursor="pointinghand")
        l3 = Label(self.frame, text="3. Movie search", font=("", 16), cursor="pointinghand")
        l4 = Label(self.frame, text="4. Actor search", font=("", 16), cursor="pointinghand")
        l5 = Label(self.frame, text=" ")
        l1.pack()
        l2.pack()
        l3.pack()
        l4.pack()
        l5.pack(pady=45)

        l1.bind("<Button-1>", self.top_movies)
        l2.bind("<Button-1>", self.top_actors)
        l3.bind("<Button-1>", self.movie_search)
        l4.bind("<Button-1>", self.actor_search)

        # Button to get back to welcome page
        b1 = Button(self.frame, text="Back to welcome page", command=self.welcome_page, width=25, height=3)
        b1.pack()

    def top_movies(self, event):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Top Movies", font=("", 40, "bold"))
        t1.pack(pady=25)

        # Creating object
        o1 = LiveData()
        top_movies = o1.get_top_movies()

        gross = list()
        gross.append(0)
        movies_name = []
        for key, movie in top_movies.items():
            gross.append(int(movie["gross"][1:].replace(",", "")))
            movies_name.append(movie["name"])

        f = Figure(figsize=(10, 10), dpi=60)
        a = f.add_subplot(111)
        ind = arange(len(movies_name)+1)  # the x locations for the groups

        a.bar(ind, gross, 0.35)
        a.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        a.set_xticklabels([""] + movies_name, rotation=90)
        a.tick_params(axis='x', pad=8)
        f.subplots_adjust(bottom=0.4)
        f.subplots_adjust(top=0.9)

        canvas = FigureCanvasTkAgg(f, master=self.frame)
        canvas.show()
        canvas.get_tk_widget().pack()

        canvas._tkcanvas.pack()

        # Button to get back to welcome page
        b1 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b1.pack(pady=20)

    def top_actors(self, event):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Top Actors", font=("", 40, "bold"))
        t1.pack(pady=25)

        # Creating object
        o1 = LiveData()
        top_actors = o1.get_top_actors()

        gross = list()
        gross.append(0)
        actors_name = []
        for key, actor in top_actors.items():
            actor_gross = float(actor["gross"][1:].replace(",", "").replace(" mi", "")) * 1000000
            gross.append(actor_gross)
            actors_name.append(actor["name"])

        f = Figure(figsize=(5, 5), dpi=115)
        a = f.add_subplot(111)
        ind = arange(len(actors_name)+1)  # the x locations for the groups

        a.bar(ind, gross, 0.35, color="r")
        a.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        a.set_xticklabels([""] + actors_name, rotation=90)
        f.subplots_adjust(bottom=0.4)
        f.subplots_adjust(top=0.9)

        canvas = FigureCanvasTkAgg(f, master=self.frame)
        canvas.show()
        canvas.get_tk_widget().pack()

        canvas._tkcanvas.pack()

        # Button to get back to welcome page
        b1 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b1.pack(pady=25)

    def movie_search(self, event, result=" "):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Movie Search", font=("", 40, "bold"))
        t1.pack(pady=25)

        t2 = Label(self.frame, text=" ")
        t2.pack(pady=25)

        self.movie = Entry(self.frame)
        b1 = Button(self.frame, text="Search", command=self.searching_movie, width=10, height=2)
        self.movie.pack()
        b1.pack()
        self.movie.focus_set()

        t3 = Label(self.frame, text=result)
        t3.pack(pady=25)

        # Button to get back to welcome page
        b2 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b2.pack()

    def actor_search(self, event, result=" "):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Actor Search", font=("", 40, "bold"))
        t1.pack(pady=25)

        t2 = Label(self.frame, text=" ")
        t2.pack(pady=25)

        self.actor = Entry(self.frame)
        b1 = Button(self.frame, text="Search", command=self.searching_actor, width=10, height=2)
        self.actor.pack()
        b1.pack()
        self.actor.focus_set()

        t3 = Label(self.frame, text=result)
        t3.pack(pady=25)

        # Button to get back to welcome page
        b2 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b2.pack()

    def searching_movie(self):
        self.movie_result = self.movie.get()
        db = DatabaseAccess()
        movie_result = db.search_movie(self.movie_result)
        if movie_result != False:
            self.movie_search_result(movie_result)
        else:
            ld = LiveData()
            movie_result = ld.get_movie(self.movie_result)
            if isinstance(movie_result, str):
                self.movie_search("", movie_result)
            else:
                db.insert_movie(movie_result)
                self.movie_search_result(movie_result)

    def searching_actor(self):
        self.actor_result = self.actor.get()
        db = DatabaseAccess()
        actor_result = db.search_actor(self.actor_result)
        if actor_result != False:
            self.actor_search_result(actor_result)
        else:
            ld = LiveData()
            actor_result = ld.get_actor(self.actor_result)
            if isinstance(actor_result, str):
                self.actor_search("", actor_result)
            else:
                db.insert_actor(actor_result)
                self.actor_search_result(actor_result)

    def movie_search_result(self, result):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Movie Search", font=("", 40, "bold"))
        t1.pack(pady=25)

        t2 = Label(self.frame, text=" ")
        t2.pack(pady=15)

        gross = list()
        gross.append(0)
        actors_name = []
        for actor in result["actors"]:
            gross.append(int(actor["gross"]))
            actors_name.append(actor["name"])

        f = Figure(figsize=(5, 5), dpi=100)
        a = f.add_subplot(111)
        ind = arange(len(actors_name)+1)  # the x locations for the groups

        a.bar(ind, gross, 0.35, color="r")
        a.set_xticks(range(0,len(actors_name)+1))
        a.set_xticklabels([""] + actors_name, rotation=90)
        f.subplots_adjust(bottom=0.4)
        f.subplots_adjust(top=0.9)

        canvas = FigureCanvasTkAgg(f, master=self.frame)
        canvas.show()
        canvas.get_tk_widget().pack()

        canvas._tkcanvas.pack()

        t3 = Label(self.frame, text=" ")
        t3.pack(pady=15)

        # Button to get back to welcome page
        b2 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b2.pack()

    def actor_search_result(self, result):
        self.frame.destroy()
        self.frame = Frame(self.master, width=1400, height=800)
        self.frame.pack()

        # Project name in the top
        t1 = Label(self.frame, text="VICE CITY - Actor Search", font=("", 40, "bold"))
        t1.pack(pady=25)

        gross = list()
        gross.append(0)
        movies_name = []
        for movie in result["movies"]:
            gross.append(int(movie["gross"]))
            movies_name.append(movie["name"])

        f = Figure(figsize=(10, 9), dpi=60)
        a = f.add_subplot(111)
        ind = arange(len(movies_name)+1)  # the x locations for the groups

        a.bar(ind, gross, 0.35, color="b")
        a.set_xticks(range(0,len(movies_name)+1))
        a.set_xticklabels([""] + movies_name, rotation=90)
        f.subplots_adjust(bottom=0.4)
        f.subplots_adjust(top=0.9)

        canvas = FigureCanvasTkAgg(f, master=self.frame)
        canvas.show()
        canvas.get_tk_widget().pack()

        canvas._tkcanvas.pack()

        t3 = Label(self.frame, text=" ")
        t3.pack(pady=15)

        # Button to get back to welcome page
        b2 = Button(self.frame, text="Back to welcome page", command=self.enter, width=25, height=3)
        b2.pack()



In [14]:
ProjectHollywood()

<__main__.ProjectHollywood instance at 0x1136c23b0>